# EEG Signal Processing Pipeline with GPU Acceleration
This notebook demonstrates a pipeline to process EEG signals from EDF files. The pipeline includes interpolation, standardization, detrending, and applying a Hidden Markov Model (HMM) for stage classification or clustering with GPU acceleration.

In [10]:
# Import Required Libraries
import numpy as np
import pandas as pd
import mne
from scipy.signal import detrend
from hmmlearn import hmm
# Import GPU libraries
import tensorflow as tf
import os
# Check if GPU is available
print(f"TensorFlow sees the following devices:\n{tf.config.list_physical_devices()}")
print(f"Is GPU available: {tf.config.list_physical_devices('GPU')}")

2025-06-04 15:30:36.962521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow sees the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
# Set up GPU memory management
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print('Memory growth set to True')
    except RuntimeError as e:
        print(e)

Memory growth set to True


## Step 1: Read EDF File
Load the EEG signal from an EDF file with a sampling frequency of 512 Hz.

In [12]:
# Load EDF file
def load_edf(file_path):
    # Use mne.io.read_raw_edf to read EDF files
    raw = mne.io.read_raw_edf(file_path, preload=True)
    # Get data as numpy array, selecting the first channel for simplicity
    # Modify this based on which channels you want to analyze
    data = raw.get_data()[0]
    return data

file_path = 'by captain borat/raw/EEG_0_per_hour_2024-03-20 17_12_18.edf'
eeg_signal = load_edf(file_path)

Extracting EDF parameters from /home/yahia/notebooks/by captain borat/raw/EEG_0_per_hour_2024-03-20 17_12_18.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


Reading 0 ... 44153855  =      0.000 ... 86237.998 secs...


## Step 2: Interpolate Signal
Interpolate locally with a threshold of 3x the standard deviation using a sliding window of 10 seconds.

In [13]:
# Interpolate signal
def interpolate_signal(signal, window_size, threshold_factor):
    interpolated_signal = signal.copy()
    std_dev = np.std(signal)
    threshold = threshold_factor * std_dev
    for i in range(0, len(signal), window_size):
        window = signal[i:i+window_size]
        outliers = np.abs(window - np.mean(window)) > threshold
        interpolated_signal[i:i+window_size][outliers] = np.mean(window)
    return interpolated_signal

window_size = 512 * 10  # 10 seconds
threshold_factor = 3
eeg_signal_interpolated = interpolate_signal(eeg_signal, window_size, threshold_factor)

## Step 3: Standardize Signal
Standardize the signal to have zero mean and unit variance.

In [14]:
# Standardize signal
def standardize_signal(signal):
    return (signal - np.mean(signal)) / np.std(signal)

eeg_signal_standardized = standardize_signal(eeg_signal_interpolated)

## Step 4: Remove Linear Trends
Remove linear trends from the signal.

In [15]:
# Remove linear trends
eeg_signal_detrended = detrend(eeg_signal_standardized)

## Step 5: Apply Hidden Markov Model (HMM) with GPU Acceleration
Use a GPU-accelerated implementation of HMM to classify or cluster different stages from the signal.

In [16]:
# GPU-accelerated HMM implementation using TensorFlow for matrix operations
class TFHMM:
    def __init__(self, n_components=5, n_iter=100):
        self.n_components = n_components
        self.n_iter = n_iter
        self.means_ = None
        self.covars_ = None
        self.transmat_ = None
        self.startprob_ = None

    def fit(self, X):
        # Convert data to TensorFlow tensor
        X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
        n_samples, n_features = X_tf.shape

        # Initialize parameters randomly
        self.means_ = tf.Variable(tf.random.normal([self.n_components, n_features]))
        self.covars_ = tf.Variable(tf.ones([self.n_components, n_features]))
        self.transmat_ = tf.Variable(tf.random.uniform([self.n_components, self.n_components]))
        self.startprob_ = tf.Variable(tf.random.uniform([self.n_components]))

        # Normalize transition and start probabilities
        self.transmat_.assign(tf.nn.softmax(self.transmat_, axis=1))
        self.startprob_.assign(tf.nn.softmax(self.startprob_))

        # Simple EM algorithm implementation
        for i in range(self.n_iter):
            # E-step: compute responsibilities (using log-likelihood for stability)
            log_resp = self._compute_log_likelihood(X_tf) + tf.math.log(self.startprob_[:, tf.newaxis])
            # Normalize responsibilities
            log_resp = log_resp - tf.reduce_logsumexp(log_resp, axis=0)[tf.newaxis, :]
            resp = tf.exp(log_resp)
            
            # M-step: update parameters
            # Update means
            numer = tf.matmul(resp, X_tf)
            denom = tf.reduce_sum(resp, axis=1)[:, tf.newaxis]
            self.means_.assign(numer / denom)
            
            # Update covariances
            centered = X_tf[tf.newaxis, :, :] - self.means_[:, tf.newaxis, :]
            squared = centered ** 2
            self.covars_.assign(tf.reduce_sum(resp[:, :, tf.newaxis] * squared, axis=1) / denom)

        return self
    
    def predict(self, X):
        # Convert data to TensorFlow tensor
        X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
        # Use Viterbi algorithm for decoding
        log_likelihood = self._compute_log_likelihood(X_tf)
        return tf.argmax(log_likelihood, axis=0).numpy()
    
    def _compute_log_likelihood(self, X):
        # Calculate log-likelihood for each state and sample
        X_expanded = X[tf.newaxis, :, :]  # Shape: [1, n_samples, n_features]
        means_expanded = self.means_[:, tf.newaxis, :]  # Shape: [n_components, 1, n_features]
        covars_expanded = self.covars_[:, tf.newaxis, :]  # Shape: [n_components, 1, n_features]
        
        # Calculate log-likelihood using Gaussian distribution
        diff = X_expanded - means_expanded  # Shape: [n_components, n_samples, n_features]
        log_likelihood = -0.5 * tf.reduce_sum(diff**2 / covars_expanded, axis=2)
        log_likelihood = log_likelihood - 0.5 * tf.reduce_sum(tf.math.log(2 * np.pi * covars_expanded), axis=2)
        return log_likelihood

def apply_gpu_hmm(signal, n_states):
    # Reshape the signal for HMM
    signal_reshaped = signal.reshape(-1, 1)
    # Apply GPU-accelerated HMM
    model = TFHMM(n_components=n_states, n_iter=100)
    model.fit(signal_reshaped)
    hidden_states = model.predict(signal_reshaped)
    return hidden_states

# Define the number of states
n_states = 5

# Apply GPU accelerated HMM
with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
    print(f"Running HMM on: {tf.config.list_logical_devices()}")
    hidden_states = apply_gpu_hmm(eeg_signal_detrended, n_states)

Running HMM on: [LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2025-06-04 15:30:44.166239: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-04 15:30:44.379233: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10446 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1


In [ ]:
# Alternative GPU-accelerated HMM using the hmmlearn library with CuPy if available
try:
    import cupy as cp
    print("CuPy is available for GPU acceleration")
    # Use CuPy for GPU acceleration
    def apply_hmm_cupy(signal, n_states):
        # Move data to GPU
        signal_gpu = cp.array(signal.reshape(-1, 1))
        # Initialize and fit the model
        model = hmm.GaussianHMM(n_components=n_states, covariance_type='diag', n_iter=100)
        model.fit(cp.asnumpy(signal_gpu))  # Convert back to numpy for hmmlearn
        # Predict states
        hidden_states = model.predict(cp.asnumpy(signal_gpu))
        return hidden_states
    
    # Apply CuPy accelerated HMM
    hidden_states_cupy = apply_hmm_cupy(eeg_signal_detrended, n_states)
    print("HMM computation completed using CuPy")
except ImportError:
    print("CuPy is not available, using TensorFlow implementation only")

CuPy is available for GPU acceleration


## Step 6: Visualize Results
Visualize the classified or clustered stages from the signal.

In [ ]:
# Visualize results
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))
plt.plot(eeg_signal_detrended, label='Detrended Signal')
plt.plot(hidden_states, label='Hidden States (GPU)', alpha=0.7)
plt.legend()
plt.title('EEG Signal and Hidden States (GPU-accelerated HMM)')
plt.show()

## Performance Analysis: CPU vs GPU
Compare the performance of CPU and GPU implementations

In [ ]:
# Performance comparison between CPU and GPU implementations
import time

# CPU implementation
def apply_hmm_cpu(signal, n_states):
    model = hmm.GaussianHMM(n_components=n_states, covariance_type='diag', n_iter=100)
    signal_reshaped = signal.reshape(-1, 1)
    model.fit(signal_reshaped)
    hidden_states = model.predict(signal_reshaped)
    return hidden_states

# Time the CPU implementation
start_time = time.time()
hidden_states_cpu = apply_hmm_cpu(eeg_signal_detrended, n_states)
cpu_time = time.time() - start_time
print(f"CPU Implementation Time: {cpu_time:.4f} seconds")

# Time the GPU implementation
start_time = time.time()
with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
    hidden_states_gpu = apply_gpu_hmm(eeg_signal_detrended, n_states)
gpu_time = time.time() - start_time
print(f"GPU Implementation Time: {gpu_time:.4f} seconds")

# Calculate speedup
if gpu_time > 0:
    print(f"Speedup: {cpu_time/gpu_time:.2f}x")

# Compare results
plt.figure(figsize=(15, 8))
plt.subplot(2, 1, 1)
plt.plot(eeg_signal_detrended[:1000], label='Signal (first 1000 points)', alpha=0.7)
plt.plot(hidden_states_cpu[:1000], label='CPU HMM', alpha=0.7)
plt.legend()
plt.title('CPU Implementation')

plt.subplot(2, 1, 2)
plt.plot(eeg_signal_detrended[:1000], label='Signal (first 1000 points)', alpha=0.7)
plt.plot(hidden_states_gpu[:1000], label='GPU HMM', alpha=0.7)
plt.legend()
plt.title('GPU Implementation')
plt.tight_layout()
plt.show()